### Understanding gradient descent

In [19]:
#gradient descent function
def gradient_descent(gradient, start, learning_rate, n_iter):
    vector = start
    for i in range(n_iter):
        step_size = learning_rate * gradient(vector)
        vector = vector - step_size
    return vector

# gradient ==> callable object
# start ==> initial point of search ==> tuple/list/numpy array/scalar
# learning_rate ==> controls magnitude of vector update
# n_iter ==> number of iterations

- If learning rate is too small, then the algorithm will converge very slowly
- If learning rate is too big, it can make the algorithm too divergent

Another termination criteria is **Tolerance**. The step size should be greater than a minimum threshold (tolerance). If the step size is too small, i.e. tends to zero, then it means that the slope or the gradient is also tending to zero. This indicates that we are close to our local minimum.

In [20]:
# gradient descent with tolerance

import numpy as np

def gradient_descent2(gradient, start, learning_rate, n_iter, tolerance):
    vector = start
    for i in range(n_iter):
        step_size = learning_rate * gradient(vector)
        # since each vector has multiple parameters, we compare all the elements of the numpy array
        # i.e. step_size for all parameters
        if np.all(np.abs(step_size) <= tolerance):
            break
        vector = vector - step_size
    return vector

**Example 1.** 

f(v) = v^2

gradient of f(v) = 2v

In [21]:
# only one parameter v
# testing the gradient_descent function on v^2
# gradient or derivative here is 2v

gradient_descent( gradient = lambda v : 2*v, start = 10.0, learning_rate = 0.1, n_iter = 100)

2.0370359763344878e-09

**Example 2.** 

f(v1, v2) = v1^2 + v2^4

gradient of f(v1, v2) = 2*v1 + 4*v2^3

In [22]:
# 2 parameters v1 and v2
# function is v1^2 + v2^4
# gradient = 2*v1 + 4*v2^3

gradient_descent2(gradient=lambda v: np.array([2 * v[0], 4 * v[1]**3]),
                  start=np.array([1.0, 1.0]),
                  learning_rate=0.2,
                  n_iter = 1000,
                  tolerance=1e-08)


array([1.41661026e-222, 2.47806975e-002])

The above example correctly converges at v1 = v2 = 0

#### Residual sum of squares (RSS)

In [23]:
# linear regression
# x(input) and y(output) are observations from the dataset
# b is the vector with all parameters

def rss_gradient(x,y,b):
    res = b[0] + b[1]*x - y
    return 2*res.mean(), 2*(res*x).mean() # returns a tuple

In [24]:
# new gradient descent with x and y inputs
# start gives the value for initally assumed values for the parameters

def gradient_descent3( gradient, x, y, start, learning_rate =0.1, n_iter = 50, tolerance = 1e-06):
    # starting point
    vector = start
    for i in range(n_iter):
        # converting output tuple from "gradient function" to array
        # enables elementwise multiplication
        step_size = learning_rate * np.array(gradient(x, y, vector))
        if(np.all(np.abs(step_size)) <= tolerance):
            break
        vector = vector - step_size
    return vector

In [25]:
x = np.array([5, 15, 25, 35, 45, 55])
y = np.array([5, 20, 14, 32, 22, 38])

In [26]:
gradient_descent3( rss_gradient, x, y, start = [0.5, 0.5], learning_rate = 0.0008, n_iter = 100000)

array([5.63333333, 0.54      ])

The parameters corresponding to the best fit line are b0 = 5.63 and b1 = 0.54

The equation for the best fit line is f(x) = 5.63 + 0.54x

##### More general function for gradient_descent

In [27]:
def gradient_descent( gradient, x, y, start, learning_rate = 0.1, n_iter = 100, tolerance = 1e-06, 
                     dtype = "float64"):
    # checking edge cases for all arguments
    
    # setting up consistent data type for all arguments
    dtype_ = np.dtype(dtype)
    
    # check if gradient is callable
    if not callable(gradient):
        raise TypeError("'gradient' must be callable!")
    
    # converting x(inputs) and y(outputs) to Numpy arrays of float datatype
    x, y = np.array(x, dtype_) , np.array(y, dtype_)
    # dimensions of x and y should be equal
    if x.shape[0] != y.shape[0]:
        raise ValueError("Dimensions of x and y should match!")
    
    # settin up learning_rate as numpy array of float datatype
    learning_rate = np.array(learning_rate, dtype = dtype_)
    if np.any(learning_rate <= 0):
        raise ValueError("'learning_rate' must be greater than zero!")
    
    # checking number of iterations
    n_iter = int(n_iter)
    if n_iter < 0:
        raise ValueError("No. of iterations should be greater than 0!")
        
    # setting up tolerance as numpy array of float datatype
    tolerance = np.array(tolerance, dtype = dtype_)
    if np.any(tolerance <= 0):
        raise ValueError("'tolerance' must be greater than zero!")
        
    vector = np.array(start, dtype = dtype_)
    
    for i in range(n_iter):
        step_size = learning_rate * np.array(gradient(x, y, vector), dtype_)
        if np.all(np.abs(step_size) <= tolerance):
            break
        vector = vector - step_size
    return vector


#### Types of Stochastic Gradient Descent

- **Online stochastic gradient descent**: is a variant in which we estimate the gradient of cost function for all observations (i.e. each **epoch**/iteration goes through the entire dataset) and updates the paramters of the cost function accordingly. Helps find the global minimum (among multiple local minimums). 

- **Batch stochastic gradient descent**: we take subsets of all observations into **mini-batches**. Gradients are calculated and the parameters of the loss functions are updated iteratively. Online stochastic gradient could be thought of as a special case of batch stochastic where there's only one batch taking all observations.

##### What happens inside the iterations?

1. Randomly divides set of observations into **mini-batches.** 
2. For each minibatch, gradient is computed and the parameter vector is updated.
3. Once all minibatches are used, it means one iteration or **epoch** is finished, move on to the next epoch. 

In [28]:
def batch_sgd( gradient, x, y, start, learning_rate = 0.1, n_iter = 100, tolerance = 1e-06, 
              dtype = "float64", batch_size = 1, random_state = None ):
    
    # gradient must be callable
    if not callable(gradient):
        raise TypeError("'gradient' must be callable!")
    
    # consistent datatypes
    dtype_ = np.dtype(dtype)
    
    # coverting x(inputs) and y(outputs) to array with datatype as above
    x, y = np.array(x, dtype = dtype_) , np.array(y, dtype = dtype_)
    # number of observations, number of inputs and number of outputs should be same
    n_obs = x.shape[0]
    if n_obs != y.shape[0]:
        raise ValueError("'x' and 'y' lengths do not match!")
    
    # concatenate the x and y into a single array
    
    # transform x into a line vector with n observations
    # transform y into a column with n observation 
    # https://stackoverflow.com/questions/57962718/reshaping-data-in-numpy-with-1-1-what-does-it-mean
    
    xy = np.c_[ x.reshape(n_obs, -1), y.reshape(n_obs, 1) ]
    
    # learning_rate
    learning_rate = np.array(learning_rate, dtype = dtype_)
    if np.any(learning_rate) <= 0:
        raise ValueError("'learning_rate' must be greater than 0!")
    
    # n_iter
    n_iter = int(n_iter)
    if n_iter <= 0:
        raise ValueError("'n_iter' must be greater than 0!")
    
    # tolerance
    tolerance = np.array(tolerance, dtype=dtype_)
    if np.any(tolerance <=0):
        raise ValueError("'tolerance' must be greater than 0!")
        
    # initializing parameter vector
    vector = np.array(start, dtype = dtype_)
    
    # intializing random number generator to set mini-batches
    seed = None if random_state is None else int(random_state)
    randnumgenerator = np.random.default_rng(seed=seed)
    
    # setting up batch size
    batch_size = int(batch_size)
    if not 0 < batch_size <= n_obs:
        raise ValueError("'batch_size' needs to be greater than zero and less than the number of oberservations")
    
# Performing batch gradient descent
    for i in range(n_iter):
        # shuffle xy to randomly select obeservations in batches for every iteration
        randnumgenerator.shuffle(xy)
        
        # cover n_obs observations, repeated for each mini batch
        for start in range(0, n_obs, batch_size):
            stop = start + batch_size
            
            # select batches from start to stop index in xy, then break into x and y batch indiviually
            x_batch = xy[start:stop, :-1] # first element of all arrays of xy
            y_batch = xy[start:stop, -1:] # last element of all arrays of xy
            
            # Calculating step_size
            
            # gradient
            grad = np.array(gradient(x_batch, y_batch, vector), dtype = dtype_)
            
            # step_size 
            step_size = learning_rate*grad
            
            # check with tolerance
            if np.all(np.abs(step_size) <= tolerance):
                break
            
            vector = vector - step_size
        
    return vector

In [30]:
# example
batch_sgd( rss_gradient, x, y, start=[0.5, 0.5], learning_rate=0.0008, 
    batch_size=3, n_iter=100_000, random_state=0 )

array([5.64931198, 0.81599011])

##### Momentum in stochastic gradient descent

Corrects the effect of learning rate for calculating the step size. The idea is to remember the previous step_size update of the parameter vector and apply it when calculating the next one. You don't exactly move the vector exactly in the direction of the negative gradient, but you also use the direction and magnitude of the previous move.

It helps to find the global minimum, from a bunch of local minimums since you do not get trapped in any particular local minimum. You are always pushed or pull out of the local minimums, until the gradient actually approaches zero.

The additional parameter used for Momentum is called the decay rate.


In [32]:
def momentum_sgd( gradient, x, y, start, learning_rate = 0.1, n_iter = 100, tolerance = 1e-06, 
              dtype = "float64", batch_size = 1, random_state = None, decay_rate = 0.0 ):
    
    # gradient must be callable
    if not callable(gradient):
        raise TypeError("'gradient' must be callable!")
    
    # consistent datatypes
    dtype_ = np.dtype(dtype)
    
    # coverting x(inputs) and y(outputs) to array with datatype as above
    x, y = np.array(x, dtype = dtype_) , np.array(y, dtype = dtype_)
    # number of observations, number of inputs and number of outputs should be same
    n_obs = x.shape[0]
    if n_obs != y.shape[0]:
        raise ValueError("'x' and 'y' lengths do not match!")
    
    # concatenate the x and y into a single array
    
    # transform x into a line vector with n observations
    # transform y into a column with n observation 
    # https://stackoverflow.com/questions/57962718/reshaping-data-in-numpy-with-1-1-what-does-it-mean
    
    xy = np.c_[ x.reshape(n_obs, -1), y.reshape(n_obs, 1) ]
    
    # learning_rate
    learning_rate = np.array(learning_rate, dtype = dtype_)
    if np.any(learning_rate) <= 0:
        raise ValueError("'learning_rate' must be greater than 0!")
    
    # n_iter
    n_iter = int(n_iter)
    if n_iter <= 0:
        raise ValueError("'n_iter' must be greater than 0!")
    
    # tolerance
    tolerance = np.array(tolerance, dtype=dtype_)
    if np.any(tolerance <=0):
        raise ValueError("'tolerance' must be greater than 0!")
        
    # initializing parameter vector
    vector = np.array(start, dtype = dtype_)
    
    # intializing random number generator to set mini-batches
    seed = None if random_state is None else int(random_state)
    randnumgenerator = np.random.default_rng(seed=seed)
    
    # setting up batch size
    batch_size = int(batch_size)
    if not 0 < batch_size <= n_obs:
        raise ValueError("'batch_size' needs to be greater than zero and less than the number of oberservations")
    
    # decay_rate
    decay_rate = np.array(decay_rate, dtype = dtype_)
    if np.any(decay_rate) < 0 or np.any(decay_rate) > 1:
        raise ValueError("'decay_rate' must be between 0 and 1!")
        
    # initialize the step_size 
    step_size = 0
    
# Performing batch gradient descent
    for i in range(n_iter):
        # shuffle xy to randomly select obeservations in batches for every iteration
        randnumgenerator.shuffle(xy)
        
        # cover n_obs observations, repeated for each mini batch
        for start in range(0, n_obs, batch_size):
            stop = start + batch_size
            
            # select batches from start to stop index in xy, then break into x and y batch indiviually
            x_batch = xy[start:stop, :-1] # first element of all arrays of xy
            y_batch = xy[start:stop, -1:] # last element of all arrays of xy
            
            # Calculating step_size
            
            # gradient
            grad = np.array(gradient(x_batch, y_batch, vector), dtype = dtype_)
            
            # step_size 
            step_size = learning_rate*grad - step_size*decay_rate
            
            # check with tolerance
            if np.all(np.abs(step_size) <= tolerance):
                break
            
            vector = vector - step_size
        
    return vector